### Database Creation and Ingestion

Converting raw excel files into dataframes and ingesting them into created db file,
In addition configuartion of a backlog to monitor and update stages of database creation for debugging if necessary

In [1]:
import pandas as pd
import os
from sqlalchemy import create_engine
import logging
import time

In [2]:
engine = create_engine('sqlite:///inventory.db')

logging.basicConfig(
    filename="../logs/ingestion_db.log",
    level=logging.DEBUG,
    format="%(asctime)s - %(levelname)s - %(message)s",
    filemode="a"
)

def ingest_db(df, table_name, engine):
    '''This function will ingest the dataframe into database table'''
    df.to_sql(table_name, con = engine, if_exists = 'replace', index = False, chunksize=10000)

def load_raw_data():
    '''This function will load CSV files as dataframe and ingest into db'''
    start = time.time()
    for file in os.listdir('data'):
        if '.csv' in file:
            df = pd.read_csv('data/'+file)
            logging.info(f'Ingesting {file} in db')
            ingest_db(df, file[:-4], engine)
    end = time.time()
    total_time = (end - start) / 60
    logging.info('------------Ingestion Complete------------')
    logging.info(f'\nTotal time taken: {total_time} minutes')

if __name__ == '__main__':
    load_raw_data()